## Generate report (flow, speed, and V/C) for links with a given functional class

In [1]:
# Highway links report (flow, V/C, and speeds) by functional class
#
import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

In [2]:
# Base directory for MoDX output for "base year" model results.
#
base_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Base directory for MoDX output for "comparison scenario" model results.
# 
comparison_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'
# 

### User input required: Path to root directory for current scenario

In [4]:
# ===>>>USER INPUT REQUIRED: <<<===
#
# 1. Supply path to root directory of scenario to use for the current run of this notebook:
# 
home_dir = base_scenario_dir

### User input required: Path to root of user's "sandbox" directory

In [5]:
# 2. Supply path to root of user's "sandbox" directory:
#
my_sandbox_dir = r'S:/my_modx_output_dir/'

### User input required: CSV file for report output

In [6]:
# 3. Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'functional_class_report_base_scenario.csv'

### User input required: Python list of functional class number(s)

In [7]:
# 4. Supply array of selected functional class number(s).
#    NOTE: Even if a report for a _single_ functionl class is desired, this value must be supplied in a list!
#
selected_fc_list = [1, 2]

In [8]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [9]:
# Load the links shapefile into a geopandas dataframe 
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG26986.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

,DIR,LENGTH,STATE,ANODE,BNODE,TAZ_ID,STREETNAME,ROUTENUMBE,SCEN_00,SCEN_00_FU,geometry
ID,,,,,,,,,,,
200648,0,0.303488,RI,160862.0,160893.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178534.494 860032.322, 178535.226 ..."
200637,0,0.051294,RI,160893.0,160894.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178594.130 860516.853, 178595.411 ..."
200654,0,0.121481,RI,160871.0,160834.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178447.503 860933.368, 178444.955 ..."
200617,0,0.107739,RI,160870.0,160871.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178531.717 860781.945, 178497.785 ..."
200671,0,0.120546,RI,160894.0,160870.0,208448,WALLUM LAKE RD,None,1,5,"LINESTRING (178592.473 860599.050, 178591.830 ..."
...,...,...,...,...,...,...,...,...,...,...,...
248633,0,0.095560,MA,0.0,0.0,1682,None,None,1,40,"LINESTRING (229294.356 878462.956, 229213.729 ..."
248634,0,0.059164,MA,0.0,0.0,1678,None,None,1,40,"LINESTRING (229293.929 878645.226, 229213.729 ..."
248636,0,0.039507,MA,0.0,0.0,1715,None,None,1,40,"LINESTRING (232802.124 874868.190, 232771.931 ..."


In [10]:
# Get the list of links with the specified functional clas(ses).
# NOTE: The attribute to query is 'SCEN_00_FU', not the most felicitous choice of attribute name!
#
filtered_gdf = links_gdf[links_gdf['SCEN_00_FU'].isin(selected_fc_list)]

In [11]:
# Get the list of IDs for the links with the specified functional clas(ses).
highway_links_list = filtered_gdf['ID'].to_list()

In [12]:
# Directory containing link flow CSVs
link_flow_dir = home_dir + 'out/'

In [13]:
# Individual link-flow CSV tables:
# For each time period, there is a separate flow CSV for autos and for trucks.
# To get the total volume for any given time period, 'Tot_Flow' columns these need to be summed.
# However, the V/C and speed data for *both* autos and trucks are reported in the CSV for autos.
# Clear?
#
am_flow_auto_fn = link_flow_dir + 'AM_MMA_LinkFlow.csv'
am_flow_truck_fn = link_flow_dir + 'AM_MMA_LinkFlow_Trucks.csv'
#
md_flow_auto_fn = link_flow_dir + 'MD_MMA_LinkFlow.csv'
md_flow_truck_fn = link_flow_dir + 'MD_MMA_LinkFlow_Trucks.csv'
#
pm_flow_auto_fn = link_flow_dir + 'PM_MMA_LinkFlow.csv'
pm_flow_truck_fn = link_flow_dir + 'PM_MMA_LinkFlow_Trucks.csv'
#
nt_flow_auto_fn = link_flow_dir + 'NT_MMA_LinkFlow.csv'
nt_flow_truck_fn = link_flow_dir + 'NT_MMA_LinkFlow_Trucks.csv'

In [14]:
# Read each of the above CSV files containing flow data into a dataframe
#
temp_am_auto_df = pd.read_csv(am_flow_auto_fn, delimiter=',')
temp_am_truck_df = pd.read_csv(am_flow_truck_fn, delimiter=',')
#
temp_md_auto_df = pd.read_csv(md_flow_auto_fn, delimiter=',')
temp_md_truck_df = pd.read_csv(md_flow_truck_fn, delimiter=',')
#
temp_pm_auto_df = pd.read_csv(pm_flow_auto_fn, delimiter=',')
temp_pm_truck_df = pd.read_csv(pm_flow_truck_fn, delimiter=',')
#
temp_nt_auto_df = pd.read_csv(nt_flow_auto_fn, delimiter=',')
temp_nt_truck_df = pd.read_csv(nt_flow_truck_fn, delimiter=',') 

In [15]:
# Filter the 8 temp "flow" dataframes to only include rows for the selected highway links
#
am_auto_df = temp_am_auto_df[temp_am_auto_df['ID1'].isin(highway_links_list)]
am_truck_df = temp_am_truck_df[temp_am_truck_df['ID1'].isin(highway_links_list)]
#
md_auto_df = temp_md_auto_df[temp_md_auto_df['ID1'].isin(highway_links_list)]
md_truck_df = temp_md_truck_df[temp_md_truck_df['ID1'].isin(highway_links_list)]
#
pm_auto_df = temp_pm_auto_df[temp_pm_auto_df['ID1'].isin(highway_links_list)]
pm_truck_df = temp_pm_truck_df[temp_pm_truck_df['ID1'].isin(highway_links_list)]
#
nt_auto_df = temp_nt_auto_df[temp_nt_auto_df['ID1'].isin(highway_links_list)]
nt_truck_df = temp_nt_truck_df[temp_nt_truck_df['ID1'].isin(highway_links_list)]
#
# NOTE: volume/capacity and speed data will be harvested from the "auto" dataframes subsequently.
#       See below.

In [16]:
# Further filter the filetered "flow" datafames to only include the columns containing 'Tot_Flow' (and 'ID1' and 'SCEN_00_FU')
# 
am_auto_vol_df = am_auto_df[['ID1', 'Tot_Flow']]
am_truck_vol_df = am_truck_df[['ID1', 'Tot_Flow']]
#
md_auto_vol_df = md_auto_df[['ID1', 'Tot_Flow']]
md_truck_vol_df = md_truck_df[['ID1', 'Tot_Flow']]
#
pm_auto_vol_df = pm_auto_df[['ID1', 'Tot_Flow']]
pm_truck_vol_df = pm_truck_df[['ID1', 'Tot_Flow']]
#
nt_auto_vol_df = nt_auto_df[['ID1', 'Tot_Flow']]
nt_truck_vol_df = nt_truck_df[['ID1', 'Tot_Flow']]

In [17]:
# Rename the 'Tot_Flow' column of each dataframe, appropriately
#
am_auto_vol_df = am_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_auto'})
am_truck_vol_df = am_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_truck'})
#
md_auto_vol_df = md_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_auto'})
md_truck_vol_df = md_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_truck'})
#
pm_auto_vol_df = pm_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_auto'})
pm_truck_vol_df = pm_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_truck'})
#
nt_auto_vol_df = nt_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_auto'})
nt_truck_vol_df = nt_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_truck'})

In [18]:
# Index all the "volume" dataframes on "ID1", in preparation for joining
#
am_auto_vol_df.set_index("ID1")
am_truck_vol_df.set_index("ID1")
#
md_auto_vol_df.set_index("ID1")
md_truck_vol_df.set_index("ID1")
#
pm_auto_vol_df.set_index("ID1")
pm_truck_vol_df.set_index("ID1")
#
nt_auto_vol_df.set_index("ID1")
nt_truck_vol_df.set_index("ID1")

,Tot_Flow_nt_truck
ID1,
6,1527.246090
10,1349.557702
11,1543.072290
159,368.536959
160,450.547338
...,...
248110,75.175776
248112,108.283627
248113,188.266365


In [19]:
# Join the "volume" dataframes
j1_df = am_auto_vol_df.join(am_truck_vol_df.set_index("ID1"), on="ID1")
#
j1_df.set_index("ID1")
j2_df = j1_df.join(md_auto_vol_df.set_index("ID1"), on="ID1")
#
j2_df.set_index("ID1")
j3_df = j2_df.join(md_truck_vol_df.set_index("ID1"), on="ID1")
#
j3_df.set_index("ID1")
j4_df = j3_df.join(pm_auto_vol_df.set_index("ID1"), on="ID1")
#
j4_df.set_index("ID1")
j5_df = j4_df.join(pm_truck_vol_df.set_index("ID1"), on="ID1")
#
j5_df.set_index("ID1")
j6_df = j5_df.join(nt_auto_vol_df.set_index("ID1"), on="ID1")
#
j6_df.set_index("ID1")
total_flow_join = j6_df.join(nt_truck_vol_df.set_index("ID1"), on="ID1")
#
total_flow_join.set_index("ID1")

,Tot_Flow_am_auto,Tot_Flow_am_truck,Tot_Flow_md_auto,Tot_Flow_md_truck,Tot_Flow_pm_auto,Tot_Flow_pm_truck,Tot_Flow_nt_auto,Tot_Flow_nt_truck
ID1,,,,,,,,
6,9205.715147,1041.784338,17462.407100,2559.455199,12093.084084,1024.714328,16698.777581,1527.246090
10,10472.471684,895.080848,15603.886702,2155.792352,9171.081931,850.782683,13744.754339,1349.557702
11,11614.515789,1107.608181,18101.582784,2770.082897,10566.028314,1128.215461,15927.759629,1543.072290
159,4052.133995,290.326121,8144.240703,761.717011,5053.951968,321.010523,7717.525088,368.536959
160,5082.927820,381.265143,10053.428860,1021.501800,6015.212922,438.093808,9476.237848,450.547338
...,...,...,...,...,...,...,...,...
248110,3087.078261,123.290400,6721.342221,470.875961,4200.586730,231.822943,5054.307734,75.175776
248112,4014.770888,177.576089,8673.473301,678.247091,5583.022194,333.927247,7327.170171,108.283627
248113,6043.515690,273.920032,9061.437396,923.259243,4755.996794,441.301795,8480.212198,188.266365


In [20]:
# Calculate the total volume (auto + truck) for each time period, and for the entire day
#
total_flow_join['Tot_Flow_am'] = total_flow_join['Tot_Flow_am_auto'] + total_flow_join['Tot_Flow_am_truck']
#
total_flow_join['Tot_Flow_md'] = total_flow_join['Tot_Flow_md_auto'] + total_flow_join['Tot_Flow_md_truck']
#
total_flow_join['Tot_Flow_pm'] = total_flow_join['Tot_Flow_pm_auto'] + total_flow_join['Tot_Flow_pm_truck']
#
total_flow_join['Tot_Flow_nt'] = total_flow_join['Tot_Flow_nt_auto'] + total_flow_join['Tot_Flow_nt_truck']
#
total_flow_join['Tot_Flow_daily'] = total_flow_join['Tot_Flow_am'] + total_flow_join['Tot_Flow_md'] + \
                                    total_flow_join['Tot_Flow_pm'] + total_flow_join['Tot_Flow_nt']

In [21]:
# Sanity check
total_flow_join

,ID1,Tot_Flow_am_auto,Tot_Flow_am_truck,Tot_Flow_md_auto,Tot_Flow_md_truck,Tot_Flow_pm_auto,Tot_Flow_pm_truck,Tot_Flow_nt_auto,Tot_Flow_nt_truck,Tot_Flow_am,Tot_Flow_md,Tot_Flow_pm,Tot_Flow_nt,Tot_Flow_daily
2,6,9205.715147,1041.784338,17462.407100,2559.455199,12093.084084,1024.714328,16698.777581,1527.246090,10247.499485,20021.862299,13117.798412,18226.023671,61613.183867
5,10,10472.471684,895.080848,15603.886702,2155.792352,9171.081931,850.782683,13744.754339,1349.557702,11367.552532,17759.679054,10021.864614,15094.312041,54243.408241
6,11,11614.515789,1107.608181,18101.582784,2770.082897,10566.028314,1128.215461,15927.759629,1543.072290,12722.123970,20871.665681,11694.243775,17470.831919,62758.865345
94,159,4052.133995,290.326121,8144.240703,761.717011,5053.951968,321.010523,7717.525088,368.536959,4342.460116,8905.957714,5374.962491,8086.062047,26709.442368
95,160,5082.927820,381.265143,10053.428860,1021.501800,6015.212922,438.093808,9476.237848,450.547338,5464.192963,11074.930660,6453.306730,9926.785186,32919.215539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107739,248110,3087.078261,123.290400,6721.342221,470.875961,4200.586730,231.822943,5054.307734,75.175776,3210.368661,7192.218182,4432.409673,5129.483510,19964.480026
107741,248112,4014.770888,177.576089,8673.473301,678.247091,5583.022194,333.927247,7327.170171,108.283627,4192.346977,9351.720392,5916.949441,7435.453798,26896.470608
107742,248113,6043.515690,273.920032,9061.437396,923.259243,4755.996794,441.301795,8480.212198,188.266365,6317.435722,9984.696639,5197.298589,8668.478563,30167.909513
107743,248114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
total_flow_join.set_index("ID1")

,Tot_Flow_am_auto,Tot_Flow_am_truck,Tot_Flow_md_auto,Tot_Flow_md_truck,Tot_Flow_pm_auto,Tot_Flow_pm_truck,Tot_Flow_nt_auto,Tot_Flow_nt_truck,Tot_Flow_am,Tot_Flow_md,Tot_Flow_pm,Tot_Flow_nt,Tot_Flow_daily
ID1,,,,,,,,,,,,,
6,9205.715147,1041.784338,17462.407100,2559.455199,12093.084084,1024.714328,16698.777581,1527.246090,10247.499485,20021.862299,13117.798412,18226.023671,61613.183867
10,10472.471684,895.080848,15603.886702,2155.792352,9171.081931,850.782683,13744.754339,1349.557702,11367.552532,17759.679054,10021.864614,15094.312041,54243.408241
11,11614.515789,1107.608181,18101.582784,2770.082897,10566.028314,1128.215461,15927.759629,1543.072290,12722.123970,20871.665681,11694.243775,17470.831919,62758.865345
159,4052.133995,290.326121,8144.240703,761.717011,5053.951968,321.010523,7717.525088,368.536959,4342.460116,8905.957714,5374.962491,8086.062047,26709.442368
160,5082.927820,381.265143,10053.428860,1021.501800,6015.212922,438.093808,9476.237848,450.547338,5464.192963,11074.930660,6453.306730,9926.785186,32919.215539
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248110,3087.078261,123.290400,6721.342221,470.875961,4200.586730,231.822943,5054.307734,75.175776,3210.368661,7192.218182,4432.409673,5129.483510,19964.480026
248112,4014.770888,177.576089,8673.473301,678.247091,5583.022194,333.927247,7327.170171,108.283627,4192.346977,9351.720392,5916.949441,7435.453798,26896.470608
248113,6043.515690,273.920032,9061.437396,923.259243,4755.996794,441.301795,8480.212198,188.266365,6317.435722,9984.696639,5197.298589,8668.478563,30167.909513


In [23]:
# Harvest the speed and volume-to-capacity ratio data from the 4 "auto" dataframes, one for each time period. (See above.)
# Note Python variable naming convention used here: "svc" == "speed and volume/capacity"
#
am_svc_df = am_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
md_svc_df = md_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
pm_svc_df = pm_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
nt_svc_df = nt_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]

In [24]:
# Sanity check
am_svc_df

,ID1,AB_Speed,BA_Speed,AB_VOC,BA_VOC
2,6,51.357848,NaN,0.813210,NaN
5,10,47.110854,NaN,0.874697,NaN
6,11,39.914089,NaN,0.976202,NaN
94,159,56.424749,NaN,0.734421,NaN
95,160,42.807982,NaN,0.917984,NaN
...,...,...,...,...,...
107739,248110,39.690273,NaN,0.428049,NaN
107741,248112,38.689758,NaN,0.558980,NaN
107742,248113,9.525290,NaN,1.278034,NaN
107743,248114,40.000001,NaN,0.000000,NaN


In [25]:
# Rename the columns of these "svc" dataframes in preparation for joining them with the speed dataframe, computed above.
#
am_svc_df = am_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_am', 
                                      'BA_Speed' : 'BA_Speed_am',
                                      'AB_VOC'   : 'AB_VOC_am', 
                                      'BA_VOC'   : 'BA_VOC_am'})
#
md_svc_df = md_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_md', 
                                      'BA_Speed' : 'BA_Speed_md',
                                      'AB_VOC'   : 'AB_VOC_md', 
                                      'BA_VOC'   : 'BA_VOC_md'})
#
pm_svc_df = pm_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_pm', 
                                      'BA_Speed' : 'BA_Speed_pm',
                                      'AB_VOC'   : 'AB_VOC_pm', 
                                      'BA_VOC'   : 'BA_VOC_pm'})
#
nt_svc_df = nt_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_nt', 
                                      'BA_Speed' : 'BA_Speed_nt',
                                      'AB_VOC'   : 'AB_VOC_nt', 
                                      'BA_VOC'   : 'BA_VOC_nt'})

In [26]:
# Sanity check
nt_svc_df

,ID1,AB_Speed_nt,BA_Speed_nt,AB_VOC_nt,BA_VOC_nt
2,6,65.978924,NaN,0.520553,NaN
5,10,67.936971,NaN,0.437351,NaN
6,11,66.534467,NaN,0.500508,NaN
94,159,63.978525,NaN,0.487552,NaN
95,160,62.037089,NaN,0.595037,NaN
...,...,...,...,...,...
107739,248110,39.985735,NaN,0.244261,NaN
107741,248112,39.890369,NaN,0.354069,NaN
107742,248113,37.616813,NaN,0.626404,NaN
107743,248114,40.000001,NaN,0.000000,NaN


In [27]:
# Per instructions from Marty on June 22, 2021:
# For a given time period, calculate the MIN of the AB_Speed and BA_Speed, and the MAX of the AB_VOC and BA_VOC.
# Basically, the idea is to flag the link direction with the most congestion.
#
am_svc_df['Speed_am'] = am_svc_df.apply(lambda x: min(x['AB_Speed_am'], x['BA_Speed_am']), axis=1)
am_svc_df['VOC_am'] = am_svc_df.apply(lambda x: max(x['AB_VOC_am'], x['BA_VOC_am']), axis=1)
#
md_svc_df['Speed_md'] = md_svc_df.apply(lambda x: min(x['AB_Speed_md'], x['BA_Speed_md']), axis=1)
md_svc_df['VOC_md'] = md_svc_df.apply(lambda x: max(x['AB_VOC_md'], x['BA_VOC_md']), axis=1)
#
pm_svc_df['Speed_pm'] = pm_svc_df.apply(lambda x: min(x['AB_Speed_pm'], x['BA_Speed_pm']), axis=1)
pm_svc_df['VOC_pm'] = pm_svc_df.apply(lambda x: max(x['AB_VOC_pm'], x['BA_VOC_pm']), axis=1)
#
nt_svc_df['Speed_nt'] = nt_svc_df.apply(lambda x: min(x['AB_Speed_nt'], x['BA_Speed_nt']), axis=1)
nt_svc_df['VOC_nt'] = nt_svc_df.apply(lambda x: max(x['AB_VOC_nt'], x['BA_VOC_nt']), axis=1)

In [28]:
# Santiy check #1
am_svc_df

,ID1,AB_Speed_am,BA_Speed_am,AB_VOC_am,BA_VOC_am,Speed_am,VOC_am
2,6,51.357848,NaN,0.813210,NaN,51.357848,0.813210
5,10,47.110854,NaN,0.874697,NaN,47.110854,0.874697
6,11,39.914089,NaN,0.976202,NaN,39.914089,0.976202
94,159,56.424749,NaN,0.734421,NaN,56.424749,0.734421
95,160,42.807982,NaN,0.917984,NaN,42.807982,0.917984
...,...,...,...,...,...,...,...
107739,248110,39.690273,NaN,0.428049,NaN,39.690273,0.428049
107741,248112,38.689758,NaN,0.558980,NaN,38.689758,0.558980
107742,248113,9.525290,NaN,1.278034,NaN,9.525290,1.278034
107743,248114,40.000001,NaN,0.000000,NaN,40.000001,0.000000


In [29]:
# Santiy check #2
md_svc_df

,ID1,AB_Speed_md,BA_Speed_md,AB_VOC_md,BA_VOC_md,Speed_md,VOC_md
2,6,48.877192,NaN,0.849458,NaN,48.877192,0.849458
5,10,55.114768,NaN,0.755270,NaN,55.114768,0.755270
6,11,46.976779,NaN,0.876599,NaN,46.976779,0.876599
94,159,52.592114,NaN,0.795559,NaN,52.592114,0.795559
95,160,37.027792,NaN,0.983032,NaN,37.027792,0.983032
...,...,...,...,...,...,...,...
107739,248110,39.242130,NaN,0.504717,NaN,39.242130,0.504717
107741,248112,36.973825,NaN,0.656261,NaN,36.973825,0.656261
107742,248113,18.351026,NaN,1.066015,NaN,18.351026,1.066015
107743,248114,40.000001,NaN,0.000000,NaN,40.000001,0.000000


In [30]:
# Santiy check #3
pm_svc_df

,ID1,AB_Speed_pm,BA_Speed_pm,AB_VOC_pm,BA_VOC_pm,Speed_pm,VOC_pm
2,6,40.173580,NaN,0.972513,NaN,40.173580,0.972513
5,10,55.055529,NaN,0.756224,NaN,55.055529,0.756224
6,11,47.025737,NaN,0.875905,NaN,47.025737,0.875905
94,159,46.349878,NaN,0.876654,NaN,46.349878,0.876654
95,160,31.120573,NaN,1.050558,NaN,31.120573,1.050558
...,...,...,...,...,...,...,...
107739,248110,38.240905,NaN,0.590988,NaN,38.240905,0.590988
107741,248112,32.644816,NaN,0.788927,NaN,32.644816,0.788927
107742,248113,19.194742,NaN,1.049724,NaN,19.194742,1.049724
107743,248114,40.000001,NaN,0.000000,NaN,40.000001,0.000000


In [31]:
# Santiy check #4
nt_svc_df

,ID1,AB_Speed_nt,BA_Speed_nt,AB_VOC_nt,BA_VOC_nt,Speed_nt,VOC_nt
2,6,65.978924,NaN,0.520553,NaN,65.978924,0.520553
5,10,67.936971,NaN,0.437351,NaN,67.936971,0.437351
6,11,66.534467,NaN,0.500508,NaN,66.534467,0.500508
94,159,63.978525,NaN,0.487552,NaN,63.978525,0.487552
95,160,62.037089,NaN,0.595037,NaN,62.037089,0.595037
...,...,...,...,...,...,...,...
107739,248110,39.985735,NaN,0.244261,NaN,39.985735,0.244261
107741,248112,39.890369,NaN,0.354069,NaN,39.890369,0.354069
107742,248113,37.616813,NaN,0.626404,NaN,37.616813,0.626404
107743,248114,40.000001,NaN,0.000000,NaN,40.000001,0.000000


In [32]:
# Index the "svc" dataframes in preparation for joining
am_svc_df.set_index("ID1")
md_svc_df.set_index("ID1")
pm_svc_df.set_index("ID1")
nt_svc_df.set_index("ID1")

,AB_Speed_nt,BA_Speed_nt,AB_VOC_nt,BA_VOC_nt,Speed_nt,VOC_nt
ID1,,,,,,
6,65.978924,NaN,0.520553,NaN,65.978924,0.520553
10,67.936971,NaN,0.437351,NaN,67.936971,0.437351
11,66.534467,NaN,0.500508,NaN,66.534467,0.500508
159,63.978525,NaN,0.487552,NaN,63.978525,0.487552
160,62.037089,NaN,0.595037,NaN,62.037089,0.595037
...,...,...,...,...,...,...
248110,39.985735,NaN,0.244261,NaN,39.985735,0.244261
248112,39.890369,NaN,0.354069,NaN,39.890369,0.354069
248113,37.616813,NaN,0.626404,NaN,37.616813,0.626404


In [33]:
# Join the speed and volume/capacity data to the volume data collected above into a single dataframe.
#
j7_df = total_flow_join.join(am_svc_df.set_index("ID1"), on="ID1")
j7_df.set_index("ID1")
#
j8_df = j7_df.join(md_svc_df.set_index("ID1"), on="ID1")
j8_df.set_index("ID1")
#
j9_df = j8_df.join(pm_svc_df.set_index("ID1"), on="ID1")
j9_df.set_index("ID1")
#
all_data_df = j9_df.join(nt_svc_df.set_index("ID1"), on="ID1")
all_data_df.set_index("ID1")

,Tot_Flow_am_auto,Tot_Flow_am_truck,Tot_Flow_md_auto,Tot_Flow_md_truck,Tot_Flow_pm_auto,Tot_Flow_pm_truck,Tot_Flow_nt_auto,Tot_Flow_nt_truck,Tot_Flow_am,Tot_Flow_md,...,AB_VOC_pm,BA_VOC_pm,Speed_pm,VOC_pm,AB_Speed_nt,BA_Speed_nt,AB_VOC_nt,BA_VOC_nt,Speed_nt,VOC_nt
ID1,,,,,,,,,,,,,,,,,,,,,
6,9205.715147,1041.784338,17462.407100,2559.455199,12093.084084,1024.714328,16698.777581,1527.246090,10247.499485,20021.862299,...,0.972513,NaN,40.173580,0.972513,65.978924,NaN,0.520553,NaN,65.978924,0.520553
10,10472.471684,895.080848,15603.886702,2155.792352,9171.081931,850.782683,13744.754339,1349.557702,11367.552532,17759.679054,...,0.756224,NaN,55.055529,0.756224,67.936971,NaN,0.437351,NaN,67.936971,0.437351
11,11614.515789,1107.608181,18101.582784,2770.082897,10566.028314,1128.215461,15927.759629,1543.072290,12722.123970,20871.665681,...,0.875905,NaN,47.025737,0.875905,66.534467,NaN,0.500508,NaN,66.534467,0.500508
159,4052.133995,290.326121,8144.240703,761.717011,5053.951968,321.010523,7717.525088,368.536959,4342.460116,8905.957714,...,0.876654,NaN,46.349878,0.876654,63.978525,NaN,0.487552,NaN,63.978525,0.487552
160,5082.927820,381.265143,10053.428860,1021.501800,6015.212922,438.093808,9476.237848,450.547338,5464.192963,11074.930660,...,1.050558,NaN,31.120573,1.050558,62.037089,NaN,0.595037,NaN,62.037089,0.595037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248110,3087.078261,123.290400,6721.342221,470.875961,4200.586730,231.822943,5054.307734,75.175776,3210.368661,7192.218182,...,0.590988,NaN,38.240905,0.590988,39.985735,NaN,0.244261,NaN,39.985735,0.244261
248112,4014.770888,177.576089,8673.473301,678.247091,5583.022194,333.927247,7327.170171,108.283627,4192.346977,9351.720392,...,0.788927,NaN,32.644816,0.788927,39.890369,NaN,0.354069,NaN,39.890369,0.354069
248113,6043.515690,273.920032,9061.437396,923.259243,4755.996794,441.301795,8480.212198,188.266365,6317.435722,9984.696639,...,1.049724,NaN,19.194742,1.049724,37.616813,NaN,0.626404,NaN,37.616813,0.626404


### Export report output

In [34]:
# Export the dataframe as a CSV file
fq_output_fn = my_sandbox_dir + csv_output_fn
all_data_df.to_csv(fq_output_fn, sep=',')